In [552]:
class Site:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def equal(self,other):
        return self.x == other.x and self.y == other.y
    
    def debug(self):
        print(f"(x,y): ({self.x},{self.y})")

In [553]:
class Edge:
    def __init__(self,x,y,quadEdge):
        self.x = x
        self.y = y
        self.Container = quadEdge
        self.Org = self.Container.Org
        self.Dest = self.Container.Dest
        self.Next = quadEdge # Pointer to the quaddge container
    
    def Rot(self): #-> Edge:
        return self.Container.Rot()

    def Onext(self): #-> QuadEdge:
        return self.Next

    def Sym(self): #-> Edge:
        return self.Container.Sym()

    def RotInverse(self): #-> Edge:
        return self.Container.RotInverse()
    
    def Lnext(self):# -> Edge:
        return self.Container.Lnext()
    
    def GetOrg(self):
        return self.Container.Org
    
    def GetDest(self):
        return self.Container.Dest

In [554]:
import numpy as np
def ccw(s1,s2,s3):
    #Calculate normaly
    xa,ya = s1.x,s1.y
    xb,yb = s2.x,s2.y
    xc,yc = s3.x,s3.y
    detN = xa*(yb-yc) - ya*(xb-xc) + (xb*yc-xc*yb)
    # #calculate with numpy
    # matrix = np.array([[xa,ya,1],[xb,yb,1],[xc,yc,1]])
    # detM = np.linalg.det(matrix)
    # print(f"detN: {detN}")
    # print(f"detM: {detM}")
    return detN > 0

def test_ccw(data,expect,func,msg=""):
    print(f"{msg}")

    result = func(*data)
    assert(expect == result)

# Test case
s1 = Site(0,0)
s2 = Site (1,1)
s3 = Site (0,1)
test_ccw([s1,s2,s3],expect=True,func=ccw,msg="Test s1,s2,s3 - Expect counter clockwise")
test_ccw([s1,s3,s2],expect=False,func=ccw,msg="Test s1,s3,s2 - Expect clockwise")

test_ccw([s2,s3,s1],expect=True,func=ccw,msg="Test s2,s3,s1 - Expect counter  clockwise")
test_ccw([s2,s1,s3],expect=False,func=ccw,msg="Test s2,s1,s3 - Expect clockwise")

test_ccw([s3,s1,s2],expect=True,func=ccw,msg="Test s3,s1,s2 - Expect counter clockwise")
test_ccw([s3,s2,s1],expect=False,func=ccw,msg="Test s3,s2,s1 - Expect clockwise")


Test s1,s2,s3 - Expect counter clockwise
Test s1,s3,s2 - Expect clockwise
Test s2,s3,s1 - Expect counter  clockwise
Test s2,s1,s3 - Expect clockwise
Test s3,s1,s2 - Expect counter clockwise
Test s3,s2,s1 - Expect clockwise


In [555]:
# is used to reprenset a subdivision S 
'''
p.92

The group of edges containing e is represented in the data structure by one
edge record e, divided into four parts e [0] through e [3]. Part e [r] corresponds
to the edge e. Rot’. See Figure 7a. A generic edge e = e. Rot’Flipf is represented
by the triplet (e r, f ), called an edge reference. We may think of this triplet as a
pointer to the “quarter-record” e[r] plus a bit f that tells whether we should look
at it from “above” or from “below.”
Each part e [r] of an edge record contains two fields, Data and Next. The
Data field is used to hold geometrical and other nontopological information
about the edge e. Rot’. This field neither affects nor is affected by the topological
operations that we will describe, so its contents and format are entirely dependent
on the application.

The Next field of e [r] contains a reference to the edge e0Rot(r)Onext. Given an
arbitrary edge reference (e, r, f ), the three basic edge functions Rot, Flip, and
Onext are given by the formulas

The quad-edge data structure contains no separate records for vertice or faces;
a vertex is implicitly defined as a ring of edges, and the standard
way to refer to it is to specify one of its outgoing edges.

The standard way of referring to a connected component of the edge structure 
is by giving one of its directed edges.
'''

class QuadEdge:
    def __init__(self,name):
        self.Name = name
        #self.Sym = self - not making any sense at the moment?
        # Lnext = Rnext = Sym ? are they pointing to self? 
       # self.Lnext = self # p.84 = eRotInverse.Onext.Rot
        #self.Rnext = self # p.84 = eRot.Onext.RotInverse
        #self.Dnext = self # p.84 = eSym.Onext.Sym
        #self.Oprev = self # p.84 = eRot.Onext.Rot
        #self.Lprev = self # p.84 = eRot.Onext.Sy,
       # self.Rprev = self # p.84 = eRot.Sym.Onext
       # self.Dprev = self # p.84 = eRot.RotInverse.Onext.RotInverse
       # self.Left = self # p.85 = eRotInverse.Org
        #self.Right = self # p.85 = eRotInverse.Org
        self.Org = None # p.85 an edge algebra as the orbit of e under Onext, that is the cyclic sequence of edges
        # orbit: <..., e, eOnext, eOnext2,...,eOnext-1,e,...>
        self.Dest = None # p.85 = e.Sym.Org
        
        self.Edges = [Edge(self.Org,self.Dest,self)]*4
        self.edge_index = 0
    
    # What do we return after Rot? self?
    def Rot(self): #-> Edge:
        rot_index = (self.edge_index+1) % len(self.Edges)
        return self.Edges[rot_index]
    
    def Onext(self): #-> QuadEdge:
        # (e[r+f].Next.Rot(f).Flip(f))
        # maybe return the Edge instead of the QuadEdge
        return self.Edges[self.edge_index].Next
    
    def Sym(self): #-> Edge:
        sym_index = (self.edge_index+2) % len(self.Edges)
        return self.Edges[sym_index]
    
    def RotInverse(self):# -> Edge:
        index = (self.edge_index+3) % len(self.Edges)
        return self.Edges[index]

    def Lnext(self): #-> Edge:
        return self.RotInverse().Onext().Rot()
    #def Oprev(self):
        # self.Rot()
        # nextQuadEdge = self.Onext()
        # return nextQuadEdge.Rot()
    
# def Rot(edge: Edge):
#     quadEdge = edge.Container
#     return quadEdge.Rot()

# def Onext(edge: Edge):
#     return edge.Next

# def Sym(edge):
#     quadEdge = edge.Container
#     return quadEdge.Sym()

# def RotInverse(edge):
#     quadEdge = edge.Container
#     return quadEdge.RotInverse()

# def Oprev(self):
#     self.Rot()
#     nextQuadEdge = self.Onext()
#     return nextQuadEdge.Rot()
    
#def Oprev(e):
    
# Onext, Oprev, Dnext, Dprev : 4 pointers
# Data fields?

In [556]:
#Helper function
'''
MakeEdge:
The first operator is denoted by e t MakeEdge[ 1. It takes no parameters,
and returns an edge e of a newly created data structure representing a subdivision
of the sphere (see Figure 9). Apart from orientation and direction, e will be the
only edge of the subdivision and will not be a loop; we have e Org # e Dest, eLeft
= e Right, e Lnext = e Rnext = e Sym, and e Onext = e Oprev = e.
'''                                           
def make_edge(name):
    return QuadEdge(name)

'''
Splice[a, b] and takes as parameters two
edges a and b, returning no value. 
This operation affects the two edge rings a Org
and b Org and, independently, the two edge rings a Left and b Left. In each case,
(a) if the two rings are distinct, Splice will combine them into one;
(b) if the two are exactly the same ring, Splice will break it in two separate
pieces;
(c) if the two are the same ring taken with opposite orientations, Splice will
Flip (and reverse the order) of a segment of that ring.

The parameters a and b determine the place where the edge rings will be cut
and joined. For the rings a Org and b Org, the cuts will occur immediately after a
and b (in counterclockwise order); for the rings aLeft and bLeft, the cut will
occur immediately before a Rot and b Rot.
'''
def splice(a: Edge,b: Edge):
    print("\n---BEGIN SPLICE---")
    alpha = a.Onext().Rot() # Onext(a).Rot() --> return the underlying edge
    beta = b.Onext().Rot() # Onext(a).Rot()--> return the underlying edge
    #Ref: L.Guibas and J. Stolfi, p.102
    #precompute right hand side
    bOnext = b.Onext() # QuadEdge
    aOnext = a.Onext() # QuadEdge
    betaOnext = beta.Onext() # QuadEdge
    alphaOnext = alpha.Onext() # QuadEdge
    # assign to the left hand
    print("\n---Before update---")
    print(f"bOnext: {bOnext.Name}")
    print(f"aOnext: {aOnext.Name}")
    print(f"betaOnext: {betaOnext.Name}")
    print(f"alphaOnext: {alphaOnext.Name}")

    a.Next = bOnext
    b.Next = aOnext
    alpha.Next = betaOnext
    beta.Next = alphaOnext

    print("\n---After update---")
    print(f"bOnext: {b.Onext().Name}")
    print(f"aOnext: {a.Onext().Name}")
    print(f"betaOnext: {beta.Onext().Name}")
    print(f"alphaOnext: {alpha.Onext().Name}")
    print("\n---END SPLICE---")
    return None

'''
The operation Connett[a, b] will add a new edge e 
connecting the destination of a to the origin of
b, in such a way that a Left = e Left = b Left after the connection is complete. 
For added convenience it will also set the Org and Dest fields of the new edge to
a.Dest and b.Org, respectively.
'''
def connect(a,b):
    qe = make_edge("C")
    qe.Org = a.GetOrg()
    qe.Dest = b.GetDest()
    e = qe.Edges[0]
    splice(e, a.Lnext())
    splice(e.Sym(), b)
    return qe

def left_of(a,b):
    return None

def right_of(a,b):
    return None

def delete_edge(e):
    splice(e, e.Oprev())
    splice(e.Sym(), e.Sym().Oprev())


In [557]:
#Divide and conquer
def delaunay_dc(S,edge_store = []):
    length_S = len(S)
    if length_S == 2:
        s1,s2 = S[0],S[1]
        qa = make_edge("A")
        edge_store.append(qa)
        qa.Org = s1
        qa.Org.debug()
        qa.Dest = s2
        #How do we return two edges if we only make one quad edge?
        # Which one is edge, which one is quad edge? Are they all edges only, or they are all quad edge
        return (qa.Edges[0],qa.Sym())
    elif length_S == 3:
        s1,s2,s3 = S[0],S[1],S[2]
        # Create edges a connecting s1 to s2 
        # and b connecting s2 to s3
        qa = make_edge("A")
        qb = make_edge("B")
        edge_store.append(qa)
        edge_store.append(qb)
        qa.Org = s1
        qa.Dest = s2
        qb.Org = s2
        qb.Dest = s3
        a = qa.Edges[0]
        b = qb.Edges[0]
        print("\nsplice(a.Sym(),b)")
        splice(a.Sym(),b)
        # a.Org = s1
        # a.Dest = s2
        # b.Org = s2
        # b.Dest = s3
        #Close the triangle
        if ccw(s1,s2,s3):
            print("\nccw(s1,s2,s3)")
            print("connect(b,a)")
            qc = connect(b,a)
            edge_store.append(qc)
            return (a,b.Sym())
        elif ccw(s1,s3,s2):
            print("\nccw(s1,s3,s2)")
            print("connect(b,a)")
            qc = connect(b,a)
            edge_store.append(qc)
            c = qc.Edges[0]
            return (c.Sym(),c)
        else:
            print("no ccw")
            return (a,b.Sym())
    # elif length_S >= 4:
    #     L,R = S[:length_S//2],S[length_S//2:]
    #     ldo,ldi = delaunay_dc(L)
    #     rdi,rdo = delaunay_dc(R)
    #     #Compute the lower common tangent of L and R
    #     while True:
    #         if left_of(rdi.Org,ldi):
    #             ldi = ldi.Lnext
    #         elif right_of(ldi.Org,rdi):
    #             rdi = rdi.Rprev
    #         else:
    #             print("Finish compute the lower common tangent of L and R")
    #             break
    #     #Create a first cross edge basel from rdi.Org to ldi.Org
    #     basel = connect(rdi.Sym,ldi)
    #     if ldi.Org == ldo.Org:
    #         ldo = basel.Sym
    #     if rdi.Org == rdo.Org:
    #         rdo = basel
    #     #Merge loop
    #     while True:
    #         #Locate the first L point (lcand.Dest) to be encoutered by the rising bubble
    #         #and delete Ledges out of basel.Dest that fail the circle test
    #         lcand = basel.Sym.Onext
    #         if valid(lcand):
    #             while incircle(basel.Dest, basel.Org, lcand.Dest, lcand.Onext.Dest):
    #                 t = lcand.Onext
    #                 delete_edge(lcand)
    #                 lcand = t
    #         #End if
    #         #Symmetrically locate the first R point to be hit, and delete R edges
    #         rcand = basel.Oprev
    #         if valid(rcand):
    #             while incircle(basel.Dest, basel.Org, rcand.Dest, rcand.Oprev.Dest):
    #                 t = rcand.Oprev
    #                 delete_edge(rcand)
    #                 rcand = t
    #         #End if
    #         #If both lcand and rcand are invalid, then basel is the upper common tangent
    #         if not valid(lcand) and not valid(rcand):
    #             print("Both lcand and rcand are invalid")
    #             break
    #         #if both are valid, then choose the approriate one using the incircle test
    #         if not valid(lcand) or (valid(rcand) and incircle(lcand.Dest, lcand.Org, rcand.Org, rcand.Dest)):
    #             #add cross edge basel from rcand.Dest to basel.Dest
    #             basel = connect(rcand, basel.Sym)
    #         else:
    #             #add cross edge basel from basel.Org to lcand.Dest
    #             basel = connect(basel.Sym, lcand.Sym)
    #         #end if
    #     #End loop
    #     return (ldo, rdo)
    #End delaunay_dc 


In [558]:
def prepare_data(S):
    if len(S) < 2:
        return S
    #1. Sort by x
    #2. Sort by y
    sort_x = sorted(S, key = lambda s: s.x)
    sorted_S = sorted(sort_x, key = lambda s: s.y)
    #3. Remove duplicate points
    i = 0
    while i < len(sorted_S)-1:
        if sorted_S[i].equal(sorted_S[i+1]):
            #delete i+1
            sorted_S.pop(i+1)
        else:
            i = i +1
    return sorted_S

# Test prepare_data
def test_prepare_data(S,expect_S,func,msg=""):
    print(f"{msg}")
    result_S = func(S)
    #print(f"result_S: {result_S}")
    assert(len(expect_S) == len(result_S))
    for i,result in enumerate(result_S):
        assert(result.equal(expect_S[i]))
#Test case
#1.
S = [Site(0,0)]
expect_S = [Site(0,0)]
test_prepare_data(S,expect_S,prepare_data,"Test with 1 site")
#2.
S = [Site(0,0),Site(0,0)]
expect_S = [Site(0,0)]
test_prepare_data(S,expect_S,prepare_data,"Test with 2 site")

#3.
S = [Site(0,0),Site(0,0),Site(0,0)]
expect_S = [Site(0,0)]
test_prepare_data(S,expect_S,prepare_data,"Test with 3 site")

#4.
S = [Site(0,0),Site(1,1),Site(2,2)]
expect_S = [Site(0,0),Site(1,1),Site(2,2)]
test_prepare_data(S,expect_S,prepare_data,"Test with 3 different sites")

#5.
S = [Site(0,0),Site(1,3),Site(1,1),Site(1,1),Site(1,2)]
expect_S = [Site(0,0),Site(1,1),Site(1,2),Site(1,3)]
test_prepare_data(S,expect_S,prepare_data,"Test with 4 different sites, 1 duplicate")

#6.
S = [Site(0,0),Site(1,3),Site(1,1),Site(1,1),Site(1,2),Site(-2,-6),Site(-2,-7)]
expect_S = [Site(-2,-7),Site(-2,-6),Site(0,0),Site(1,1),Site(1,2),Site(1,3)]
test_prepare_data(S,expect_S,prepare_data,"Test with 6 different sites, 1 duplicate")

Test with 1 site
Test with 2 site
Test with 3 site
Test with 3 different sites
Test with 4 different sites, 1 duplicate
Test with 6 different sites, 1 duplicate


In [559]:
# Test with 2
S = [Site(0,0),Site(1,1)]
sorted_S = prepare_data(S)
edge_store = []
e20,e21 = delaunay_dc(sorted_S,edge_store)

(x,y): (0,0)


In [560]:
print(edge_store)

In [561]:
e20.GetOrg().debug()
e20.GetDest().debug()
e21.GetOrg().debug()
e21.GetDest().debug()

(x,y): (0,0)
(x,y): (1,1)
(x,y): (0,0)
(x,y): (1,1)


In [562]:
# Test with 3
S = [Site(0,0),Site(1,1),Site(1,0)]
sorted_S = prepare_data(S)
edge_store = []
e1,e2 = delaunay_dc(sorted_S, edge_store)
print(edge_store)


splice(a.Sym(),b)

---BEGIN SPLICE---

---Before update---
bOnext: B
aOnext: A
betaOnext: B
alphaOnext: A

---After update---
bOnext: A
aOnext: B
betaOnext: A
alphaOnext: B

---END SPLICE---

ccw(s1,s2,s3)
connect(b,a)

---BEGIN SPLICE---

---Before update---
bOnext: B
aOnext: C
betaOnext: A
alphaOnext: C

---After update---
bOnext: C
aOnext: A
betaOnext: C
alphaOnext: A

---END SPLICE---

---BEGIN SPLICE---

---Before update---
bOnext: C
aOnext: A
betaOnext: A
alphaOnext: C

---After update---
bOnext: A
aOnext: C
betaOnext: C
alphaOnext: A

---END SPLICE---
[<__main__.QuadEdge object at 0x000001E081BBEAE0>, <__main__.QuadEdge object at 0x000001E081BBE0C0>, <__main__.QuadEdge object at 0x000001E081A9C170>]


In [563]:
for edge in edge_store:
    print(f"Edge: {edge.Name}")
    edge.Org.debug()
    edge.Dest.debug()

Edge: A
(x,y): (0,0)
(x,y): (1,0)
Edge: B
(x,y): (1,0)
(x,y): (1,1)
Edge: C
(x,y): (1,0)
(x,y): (1,0)


In [564]:
for s in sorted_S:
    s.debug()

(x,y): (0,0)
(x,y): (1,0)
(x,y): (1,1)


In [565]:
begin = e1.Container
next = e1.Onext()
print(begin.Name)
counter = 0
print(begin.Name)
print(f"org: {begin.Org.debug()}")
print(f"dest: {begin.Org.debug()}")
while begin != next:
    print(next.Name)
    print(f"org: {next.Org.debug()}")
    print(f"dest: {next.Org.debug()}")
    next = next.Onext()
    if counter == 4:
        break
    counter += 1

A
A
(x,y): (0,0)
org: None
(x,y): (0,0)
dest: None


In [566]:
e1.Org.debug()

AttributeError: 'NoneType' object has no attribute 'debug'

In [ ]:
e2.Org.debug()

In [ ]:
e2.Container

In [ ]:
current = e2.Container
next = e2.Onext()
counter = 0
print(current.Name)
while current != next:
    print(next.Name)
    next = next.Onext()
    if counter == 4:
        break
    counter += 1

In [ ]:
e1.Container

In [ ]:
e1.Next

In [ ]:
e1.Container.Onext()

In [ ]:
# Test with 4
#S = [Site(0,0),Site(1,1),Site(1,0),Site(0,4)]
#delaunay_dc(S)